<h1>Oracle执行计划<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#定义" data-toc-modified-id="定义-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>定义</a></span></li><li><span><a href="#查看执行计划" data-toc-modified-id="查看执行计划-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>查看执行计划</a></span><ul class="toc-item"><li><span><a href="#Autotrace" data-toc-modified-id="Autotrace-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Autotrace</a></span></li></ul></li></ul></div>

## 定义

&emsp;&emsp;执行计划是一条查询语句在Oracle中的执行过程或访问路径的描述。

![执行计划示例](https://raw.githubusercontent.com/Henry-chr/Uplearning/main/Read/Oracle/knowledge/image/%E6%89%A7%E8%A1%8C%E8%AE%A1%E5%88%92%E7%A4%BA%E4%BE%8B.png)

执行计划的三个部分：
1. 目标SQL的正文、sql_id和执行计划对应的plan_hash_value
![SQL的正文](https://gitee.com/henry-chr/Uplearning/raw/main/Read/Oracle/knowledge/image/%E7%9B%AE%E6%A0%87SQL%E6%AD%A3%E6%96%87.png)

2. 执行计划主体，主要有内部执行步骤、执行顺序、谓词信息、列信息、Cardinality、Cost等
![执行计划主体](https://gitee.com/henry-chr/Uplearning/raw/main/Read/Oracle/knowledge/image/%E6%89%A7%E8%A1%8C%E8%AE%A1%E5%88%92%E4%B8%BB%E4%BD%93.png)

3. 执行计划的额外补充信息，是否动态采用（dynamic sampling）、是否Cardinality Feedback、是否SQL Profile
![额外补充信息](https://gitee.com/henry-chr/Uplearning/raw/main/Read/Oracle/knowledge/image/%E6%89%A7%E8%A1%8C%E8%AE%A1%E5%88%92%E8%A1%A5%E5%85%85%E4%BF%A1%E6%81%AF.png)

## 查看执行计划

### Autotrace